## Required Packages

To run the tests and main components of this project, make sure you have the following packages installed in your environment:

```bash
pip install \
    asyncio \
    typing-extensions \
    pydantic \
    autogen-core \
    autogen-agentchat \
    autogen-ext[ollama]

In [2]:
import asyncio
from typing import Literal
from autogen_core.models import UserMessage, ModelInfo
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from pydantic import BaseModel
import time

counter = 0

# Basic example
This example shows how to create an agent with autogen API.

In [7]:
model_based  = "hf.co/stepii/salamandra-7b-instruct-tools-GGUF:Q8_0"
model_client = OllamaChatCompletionClient(
    model = model_based,
    model_info=ModelInfo(vision=False, function_calling=True, json_output=False, family="unknown", structured_output=True),
)

In [8]:
async def main() -> None:
    agent = AssistantAgent(name="assistant", model_client=model_client)
    result = await agent.run(task="Name two cities in North America.")
    print(result)

if __name__ == "__main__":
    await main()

messages=[TextMessage(id='3e6c87f6-a324-4297-b3e6-77fbc5434a34', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 31, 7, 41, 59, 948437, tzinfo=datetime.timezone.utc), content='Name two cities in North America.', type='TextMessage'), TextMessage(id='ad79ff86-ad84-4f6d-b1e0-245a86ff64a1', source='assistant', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=13), metadata={}, created_at=datetime.datetime(2025, 7, 31, 7, 42, 43, 628208, tzinfo=datetime.timezone.utc), content='Two cities in North America are New York City and Toronto.', type='TextMessage')] stop_reason=None


# Agent stream example
This example shows as how to create an agent capable to generate answers in real time.


In [8]:
async def main():
    agent = AssistantAgent(
            name="assistant",
            model_client=model_client,
            model_client_stream=True,
        )

    stream = agent.run_stream(task="Name two cities in North America.")
    async for message in stream:
        print(message)


if __name__ == "__main__":
    await main()

id='14f3dd5c-062b-4511-82c2-666fc2f5139b' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 30, 8, 18, 43, 466117, tzinfo=datetime.timezone.utc) content='Name two cities in North America.' type='TextMessage'
id='c4f4286d-f5ef-4ec0-bb2b-a2448df53234' source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 30, 8, 18, 43, 939848, tzinfo=datetime.timezone.utc) content='S' full_message_id='e1c4e943-38bb-42bc-8a2d-734e1e881207' type='ModelClientStreamingChunkEvent'
id='5d3d728a-5dfc-4e90-af59-f138301c7858' source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 30, 8, 18, 44, 294682, tzinfo=datetime.timezone.utc) content='ure' full_message_id='e1c4e943-38bb-42bc-8a2d-734e1e881207' type='ModelClientStreamingChunkEvent'
id='13fa429a-176c-4a3f-a1a3-5cb2e00ec6b2' source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 30, 8, 18, 44, 745478, tzinfo=datetime.timezone.utc) co

# Example tool agents
This example shows as how to create agents that can generate responses with tools. Below you will see all the basic functions that we will be using for the rest of the basic testing.

In [9]:
async def sum_function(x: int, y: int) -> str:
    """This function grabs two numerical values in the context and aggregate them"""
    return(f"{x} + {y} = {x+y}")

async def product_function(x: int, y: int) -> str:
    """This function grabs two numerical values in the context and multiply them"""
    return(f"{x} * {y} = {x*y}")

async def plus_counter() -> str:
    """This function returns a counter plus one"""
    global counter
    counter+=1
    return(f"Counter incremented to: {counter}")

async def get_counter(counter: int) -> str:
    return(f"The current value of the counter is {counter}")

async def is_pair(value: int) -> str:
    return "Pair" if value % 2 == 0 else "Odd"

In [10]:
async def main():    
    agent = AssistantAgent(
        name="assistant", 
        model_client=model_client, 
        tools=[sum_function, product_function], 
    )

    await Console(agent.run_stream(task="What is 2 + 2?"))
    await Console(agent.run_stream(task="How much is  32 times 2?"))

if __name__ == "__main__":
    await main()

---------- TextMessage (user) ----------
What is 2 + 2?
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='0', arguments='{"x": "2", "y": "2"}', name='sum_function')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='2 + 2 = 4', name='sum_function', call_id='0', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
2 + 2 = 4
---------- TextMessage (user) ----------
How much is  32 times 2?
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='1', arguments='{"x": "32", "y": "2"}', name='product_function')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='32 * 2 = 64', name='product_function', call_id='1', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
32 * 2 = 64


In [15]:
async def main():    
    agent = AssistantAgent(
        name="assistant", 
        model_client=model_client, 
        tools=[get_counter, plus_counter], 
        max_tool_iterations = 5,
    )

    await Console(agent.run_stream(task="Can you increment the counter by 3 and show me the final result?"))
    await Console(agent.run_stream(task="Can you increment the counter by 5 and show me the final result?"))
    await Console(agent.run_stream(task="Can you increment the counter by 6 and show me the final result?"))

if __name__ == "__main__":
    await main()

---------- TextMessage (user) ----------
Can you increment the counter by 3 and show me the final result?
---------- ThoughtEvent (assistant) ----------
", "parameters": {"counter": "3"}}
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='0', arguments='{}', name='plus_counter')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='Counter incremented to: 1', name='plus_counter', call_id='0', is_error=False)]
---------- TextMessage (assistant) ----------
 TERMINATE
---------- TextMessage (user) ----------
Can you increment the counter by 5 and show me the final result?
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='1', arguments='{"counter": 6}', name='get_counter')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='The current value of the counter is 6', name='get_counter', call_id='1', is_error=False)]
---------- TextMessage (assistant) ----------
 TERMINATE


In [14]:
class AgentResponse(BaseModel):
    message: str
    response: Literal["Pair", "Odd"]

# This test doesn't work correctly if we use aina instead use another model like llama3.1:8b that also support tools
model_based  = "llama3.1:8b"
model_client = OllamaChatCompletionClient(
    model = model_based,
    model_info=ModelInfo(vision=False, function_calling=True, json_output=False, family="unknown", structured_output=True),
)
async def main():
    tool = FunctionTool(is_pair, description="Pair Analysis", strict=True)
    agent = AssistantAgent(
        name="assistant",
        model_client=model_client, 
        tools=[tool], 
        system_message="You are a system capable to detect if a number is pair or odd by a given input.",
        output_content_type=AgentResponse,
    )

    await Console(agent.run_stream(task="Is 2 an even number?")) 
    await Console(agent.run_stream(task="Is 3 an odd number?")) 

if __name__ == "__main__":
    await main()

---------- TextMessage (user) ----------
Is 2 an even number?
---------- StructuredMessage[AgentResponse] (assistant) ----------
{"message":"{'name': 'is_pair', 'parameters': {'value': 2}}","response":"Pair"}
---------- TextMessage (user) ----------
Is 3 an odd number?
---------- StructuredMessage[AgentResponse] (assistant) ----------
{"message":"{'name': 'is_pair', 'parameters': {'value': 3}}","response":"Odd"}
